In [1]:
import elasticsearch
import unicodedata
import csv
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q
import networkx as nx
import pandas as pd
import pickle

es = elasticsearch.Elasticsearch(["149.165.170.209:9200"])
#res = Search(using=es, index="filebeat*").query("match", username="root")
res = Search(using=es, index="packetbeat*")\
        .query('bool', filter=Q('exists', field='ip') & Q('exists', field='port')\
               & Q('exists', field='client_ip') & Q('exists', field='client_port')\
               & Q('exists', field='type') & Q('exists', field='method') & Q('exists', field='status')\
               & Q('exists', field='bytes_in') & Q('exists', field='bytes_out')
              )


#    .query("match", ip="/[0-9]{2,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}/")
response = res.execute()
#filtered=res.filter("exists", field='ip' & "exists", field='client_ip').count()
#print(filtered)

print("Filtered logs: %i" %res.count()) #There is an error on the number of matched logs
print("Total number of logs: %i" %response.hits.total)

#h = response.hits[0]
#print('/%s/%s/%s returned with score %f' % (
#    h.meta.index, h.meta.doc_type, h.meta.id, h.meta.score))

#df = pd.DataFrame((d.to_dict() for d in res))

Filtered logs: 15626969
Total number of logs: 15626969


In [2]:
df_tcp_udp = pd.DataFrame(((hit["@timestamp"],hit['type'],hit['method'],hit['direction'],hit['status'],hit['ip'],hit['port'],hit['client_ip'],hit['client_port'],hit['bytes_in'],hit['bytes_out']) for hit in res.scan()),\
                    columns=['Timestamp','type','method','direction','status','ip','port','client_ip','client_port','bytes_in','bytes_out'])

In [3]:
df_tcp_udp.to_pickle("tcp_udp_dataframe.pkl")

In [4]:
es = elasticsearch.Elasticsearch(["149.165.170.209:9200"])
#res = Search(using=es, index="filebeat*").query("match", username="root")
res = Search(using=es, index="packetbeat*")\
        .query('bool', filter=Q('exists', field='ip')\
               & Q('exists', field='client_ip')\
               & Q('exists', field='type') & Q('exists', field='status')\
               & Q('exists', field='bytes_in') & Q('exists', field='bytes_out')\
               & Q('match', type='ICMP')
              )

response = res.execute()

print("Filtered logs: %i" %res.count()) #There is an error on the number of matched logs
print("Total number of logs: %i" %response.hits.total)

Filtered logs: 119442
Total number of logs: 119442


In [5]:
df_icmp = pd.DataFrame(((hit["@timestamp"],hit['type'],hit['status'],hit['ip'],hit['client_ip'],hit['bytes_in'],hit['bytes_out']) for hit in res.scan()),\
                    columns=['Timestamp','type','status','ip','client_ip','bytes_in','bytes_out'])


In [6]:
df_icmp.to_pickle("icmp_dataframe.pkl")

In [8]:
df_icmp[-10:]

,Timestamp,type,status,ip,client_ip,bytes_in,bytes_out
119432,2018-11-05T23:21:57.999Z,icmp,OK,10.1.1.39,206.117.25.89,4,4
119433,2018-11-05T21:52:14.122Z,icmp,OK,10.1.1.32,23.105.70.70,6,6
119434,2018-11-05T23:41:23.157Z,icmp,OK,10.1.1.53,206.117.25.89,4,4
119435,2018-11-05T21:10:16.210Z,icmp,OK,10.1.1.10,195.169.125.251,4,4
119436,2018-11-05T23:21:57.999Z,icmp,OK,10.1.1.39,206.117.25.89,4,4
119437,2018-11-05T23:46:42.053Z,icmp,OK,10.1.1.40,203.178.148.19,4,4
119438,2018-11-05T23:41:23.157Z,icmp,OK,10.1.1.53,206.117.25.89,4,4
119439,2018-11-05T23:43:12.441Z,icmp,OK,10.1.1.18,52.11.145.86,8,8
119440,2018-11-05T23:10:57.743Z,icmp,OK,10.1.1.39,206.117.25.89,4,4
119441,2018-11-05T23:46:42.053Z,icmp,OK,10.1.1.40,203.178.148.19,4,4


In [9]:
df_tcp_udp[-10:]

,Timestamp,type,method,direction,status,ip,port,client_ip,client_port,bytes_in,bytes_out
15626959,2018-11-05T23:41:26.017Z,http,GET,out,OK,169.254.169.254,80,10.1.1.24,59850,110,1958
15626960,2018-11-05T23:02:54.783Z,http,GET,out,OK,169.254.169.254,80,10.1.1.25,35640,110,1958
15626961,2018-11-05T23:42:19.245Z,http,GET,out,OK,169.254.169.254,80,10.1.1.25,35842,110,1958
15626962,2018-11-05T23:02:53.564Z,http,GET,out,OK,169.254.169.254,80,10.1.1.25,35638,110,1958
15626963,2018-11-05T23:42:19.245Z,http,GET,out,OK,169.254.169.254,80,10.1.1.25,35842,110,1958
15626964,2018-11-05T23:42:20.430Z,dns,QUERY,out,OK,10.1.1.3,53,10.1.1.25,41094,46,62
15626965,2018-11-05T23:42:20.430Z,dns,QUERY,out,OK,10.1.1.3,53,10.1.1.25,41094,46,46
15626966,2018-11-05T23:46:20.710Z,http,GET,out,OK,169.254.169.254,80,10.1.1.25,35858,110,1958
15626967,2018-11-05T23:52:34.027Z,http,GET,out,OK,169.254.169.254,80,10.1.1.28,49898,110,1955
15626968,2018-11-05T21:54:54.199Z,http,GET,out,OK,169.254.169.254,80,10.1.1.45,34518,110,1955
